In [1]:
import sys
from pathlib import Path

sys.path.append(str(Path('../').resolve()))
assert 'python-nostr' in [Path(path).name for path in sys.path]

# client
instantiating a basic client

In [2]:
import ssl

from client import Client, ConnectedClient
from nostr.key import PrivateKey

pk_hex = PrivateKey().hex()
client = Client(private_key_hex=pk_hex)

In [4]:


with ConnectedClient(ssl_options={'cert_reqs': ssl.CERT_NONE},
                     private_key_hex=pk_hex) as connected_client:
    print(f'loaded public key is {connected_client.public_key.bech32()}\n')
    for relay in connected_client.relay_manager:
        assert relay.ws.sock is not None
        print(f'connection to {relay.url} is alive!')

for relay in connected_client.relay_manager:
    assert relay.ws.sock is None
    print(f'connection to {relay.url} is dead.')

loaded public key is npub15nj7hnx4xwanpsqdf5uwvv5y0qua4mxujx0ledcw6z9ashhqjlrqhynpse

connection to wss://nostr-2.zebedee.cloud is alive!
connection to wss://nostr.zebedee.cloud is alive!
connection to wss://nostr-relay.lnmarkets.com is alive!
connection to wss://nostr-2.zebedee.cloud is dead.
connection to wss://nostr.zebedee.cloud is dead.
connection to wss://nostr-relay.lnmarkets.com is dead.
